# Package Development (Quick Start)

This covers the basics of creating and developing a Julia package.
For more details see the [`Pkg` docs](https://pkgdocs.julialang.org/v1/).

*__NOTE__* : From the Julia REPL, you can enter the Pkg REPL by pressing `]` (and return to the Julia REPL with backspace). This allows typing commands instead of functions. We will show both representations for the operations covered here. The function version (actual Julia code) can be executed.

*__WARNING__* : This notebook is for reference. It will work if run linearly from the beginning. But if you start over within the same session, you will have problems.

It might make more sense to perform the operations in a terminal and a text editor.

## Creating a package

When you create a package, you also create a dependency environment specific to your project. We'll look at this step-by-step.


### Generate a package ("project")
```
(@v1.11) pkg> generate MyPack
```

In [ ]:
import Pkg
Pkg.generate("MyPack")

### Package status
```
(@v1.11) pkg> status
```

In [ ]:
Pkg.status()

Note that we are actually in the `julia-basics` environment. By default at the terminal you would start in the default Julia (@v1.11 when this was written) environment.

### Activate package environment
```
(@v1.11) pkg> activate MyPack
    Activating project at ...
(MyPack) pkg> status
```

In [ ]:
Pkg.activate("MyPack")

In [ ]:
Pkg.status()

### Add dependencies

Notice how the project environment is empty, that is, there are no packages available to use. To use packages within your environment, you must add them as dependencies.

```
(MyPack) pkg> add ArgParse YAML StaticArrays
```

In [ ]:
Pkg.add(["ArgParse", "YAML", "StaticArrays"])

In [ ]:
Pkg.status()

Project information is kept in the `Project.toml` file. Let's take a look.

In [ ]:
function catFile(fname::AbstractString)
    for line in readlines(fname)
        println(line)
    end
end

In [ ]:
catFile("MyPack/Project.toml")

### Write some code

The top-level module associated with a project has the same name as the package. This was already generated for you in `src/MyPack.jl`:

In [ ]:
catFile("MyPack/src/MyPack.jl")

In [ ]:
import MyPack
MyPack.greet()

Let's replace the code in `src/MyPack.jl` to do something different.

In [ ]:
write("MyPack/src/MyPack.jl", """
    module MyPack

    function modifyArg(arg::AbstractString)
        return "Hello " * arg
    end

    function modifyArg(arg::Number)
        return arg + 5
    end
    
    end
    """)

In [ ]:
MyPack.modifyArg("William")

In [ ]:
MyPack.modifyArg(23)

## Testing

In order for your package to be testable by the `Pkg` system, you must follow a convention. You need a `test` directory containing a `Project.toml` for your testing environment and a `runtests.jl` script as the entry point for your tests.

The first two requirements can be handled from `Pkg`:
```
(MyPack) pkg> activate test
(test) pkg> add Test
```
Here we generate the testing environment (and directory) and add the `Test` package as a testing dependency.

In [ ]:
Pkg.activate("MyPack/test")
Pkg.add("Test")

In [ ]:
catFile("MyPack/test/Project.toml")

The built-in `Test` package includes basic tools for testing. We'll write a quick test using `@testset`, `@test`, and `@test_throws`:

In [ ]:
write("MyPack/test/runtests.jl", """
    import MyPack
    using Test

    @testset "reportArg" begin
        @test MyPack.modifyArg(42) == 47
        @test MyPack.modifyArg("Frank") == "Hello Frank"
    
        @test_throws MethodError MyPack.modifyArg((1,2))
    end
    """)

Let's reactivate the project environment (since the `test` is currently active) and run the tests
```
(test) pkg> activate MyPack
(MyPack) pkg> test
```

In [ ]:
Pkg.activate("MyPack")

In [ ]:
Pkg.test()

`Pkg.test()` sets up a clean environment for testing (in a temp directory, separate from the project or test environment). It automatically has `MyPack` as a dependency and includes whatever is in `test/Project.toml` (in this case, just `Test`).